In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/thesis-data/Nebuli_data.csv
/kaggle/input/thesis-data/preprocessed_nlp_data.csv


In [2]:
import pandas as pd
import numpy as np
from transformers import pipeline

 


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
data_df = pd.read_csv('/kaggle/input/thesis-data/Nebuli_data.csv',index_col=0)
candidate_labels = list(data_df['parent_title'].unique())
data_df = data_df.head(100)
print(data_df.shape)
data_df.head()


(100, 11)


,id,title,introtext,fulltext,cat_id,class_2,level,cat_parent_id,parent_title,grandparent_id,grandparent_title
0,201,SGI Launches Open Storage Initiative,Silicon Graphics has today announced the Open ...,Silicon Graphics has today announced the Open ...,1836,Data Management,3,2100,Big Data,2096,Channels
1,270,IBM Releases Next Generation Storage Virtualis...,IBM has revealed its TotalStorage SAN File Sys...,IBM has revealed its TotalStorage SAN File Sys...,1836,Data Management,3,2100,Big Data,2096,Channels
2,293,IBM Launches New Mainframe and Storage Systems...,Celebrating the 40th anniversary of the IBM Sy...,Celebrating the 40th anniversary of the IBM S...,1836,Data Management,3,2100,Big Data,2096,Channels
3,337,Oracle 10g database sets the pace for a world ...,Oracle is setting the pace for the highest per...,Tests were performed on an eight-node cluster...,1836,Data Management,3,2100,Big Data,2096,Channels
4,339,IBM Releases New Automated DB2 Database Software,IBM has today announced the release of its new...,The new database software promises to deliver...,1836,Data Management,3,2100,Big Data,2096,Channels


In [4]:
list(data_df['parent_title'].unique())

['Big Data', 'Nanotechnology', 'Alternative Energy', 'Bio-IT']

In [5]:
data_df.columns

Index(['id', 'title', 'introtext', 'fulltext', 'cat_id', 'class_2', 'level',
       'cat_parent_id', 'parent_title', 'grandparent_id', 'grandparent_title'],
      dtype='object')

In [6]:
data_df.shape

(100, 11)

In [7]:
data_df = data_df[data_df['fulltext'].isnull()==False]
print(data_df.shape)

(99, 11)


In [8]:
data_df.columns

Index(['id', 'title', 'introtext', 'fulltext', 'cat_id', 'class_2', 'level',
       'cat_parent_id', 'parent_title', 'grandparent_id', 'grandparent_title'],
      dtype='object')

In [9]:
# %%time
# classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
# res = classifier(list(data_df['fulltext']), candidate_labels,multi_label = True)
# # print(type(res))
# df = pd.DataFrame(res) 
# df

In [10]:
%%time
i=0
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
def classify(seq):
    global i
    i+=1
    print(i)
    res = classifier(list(seq['fulltext']), candidate_labels,multi_label = True)
    df = pd.DataFrame({'labels':res[0]['labels'],'scores':res[0]['scores']})
    return df
    
df = data_df.groupby(['id','fulltext','parent_title']).apply(classify).reset_index()
df.head()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
CPU times: user 4h 54min 20s, sys: 9min 37s, total: 5h 3min 58s
Wall time: 2h 32min 30s


,id,fulltext,parent_title,level_3,labels,scores
0,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,0,Clinical Research,0.891404
1,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,1,Cancer Research,0.800621
2,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,2,Channels,0.671475
3,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,3,Big Data,0.519628
4,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,4,Disease Processes,0.456901


In [11]:
df.to_csv('bart_output.csv')

In [12]:
%%time
i=0
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")
def classify(seq):
    global i
    i+=1
    print(i)
    res = classifier(list(seq['fulltext']), candidate_labels,multi_label = True)
    df = pd.DataFrame({'labels':res[0]['labels'],'scores':res[0]['scores']})
    return df
    
df = data_df.groupby(['id','fulltext','parent_title']).apply(classify).reset_index()
df.head()

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
CPU times: user 1h 22min 16s, sys: 6.64 s, total: 1h 22min 23s
Wall time: 41min 20s


,id,fulltext,parent_title,level_3,labels,scores
0,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,0,Bio-IT,0.904146
1,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,1,Healthcare Technology,0.883697
2,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,2,Big Data,0.832033
3,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,3,Clinical Research,0.820768
4,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,4,Cancer Research,0.615717


In [13]:
df.to_csv('deberta_output.csv')

In [14]:
%%time
i=0
classifier = pipeline("zero-shot-classification", model="bert-large-uncased")
def classify(seq):
    global i
    i+=1
    print(i)
    res = classifier(list(seq['fulltext']), candidate_labels,multi_label = True)
    df = pd.DataFrame({'labels':res[0]['labels'],'scores':res[0]['scores']})
    return df
    
df = data_df.groupby(['id','fulltext','parent_title']).apply(classify).reset_index()
df.head()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
CPU times: user 2h 57min 5s, sys: 10.1 s, total: 2h 57min 15s
Wall time: 1h 29min 1s


,id,fulltext,parent_title,level_3,labels,scores
0,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,0,Neuroscience,0.640156
1,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,1,Stem Cells,0.624004
2,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,2,Astrobiology,0.623580
3,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,3,Bio-IT,0.603999
4,1,ACORRN (the Academic Clinical Oncology &amp; R...,Bio-IT,4,Healthcare Technology,0.600967


In [15]:
df.to_csv('bert_output.csv')

In [16]:
pip freeze | grep transformers

transformers==4.30.2
Note: you may need to restart the kernel to use updated packages.
